# Logging Custom Images

**Welcome**

To the third part of EncoderMap's customization notebooks.

Run this notebook on Google Colab:

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AG-Peter/encodermap/blob/main/tutorials/notebooks_customization/03_custom_images.ipynb)

Find the documentation of EncoderMap:

https://ag-peter.github.io/encodermap

### For Google colab only:

If you're on Google colab, please uncomment these lines and install EncoderMap.

In [ ]:
# !wget https://gist.githubusercontent.com/kevinsawade/deda578a3c6f26640ae905a3557e4ed1/raw/b7403a37710cb881839186da96d4d117e50abf36/install_encodermap_google_colab.sh
# !sudo bash install_encodermap_google_colab.sh

**Goals:**

In this tuorial you will learn how to add custom images to the "Images" section in TensorBoard. This can be done in two ways:

- [Providing `EncoderMap.add_images_to_tensorboard()` with custom function.](#logging-via-a-custom-functions)
- [Writing a custom Callback, that inherits from `encodermap.callbacks.EncoderMapBaseCallback`.](#writing-custom-callbacks)

As usual, we will start to import some packages. Along the usual packages we import the built-in package `io`.

In [ ]:
import numpy as np
import encodermap as em
import tensorflow as tf
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

import io

%matplotlib inline

We will use `io` to write a png-file to a buffer (not to disk) and provide that puffer to Tensorboard for visualization. But first, let us think about what to plot.

<a id="logging-via-a-custom-function"></a>

## Logging via a custom function

What shall we use as an example in this section?
Let's take the images, that EncoderMap automatically logs during training. These images are generated from a subset of the training data. This subset is passed through the encoder part of the network. The histogram is created from the point in the `_gen_hist_matplotlib()` function in the `encoderamp.misc.summaries.py` module.

In [ ]:
from encodermap.misc.summaries import _gen_hist_matplotlib
import inspect
print(inspect.getsource(_gen_hist_matplotlib))

We can see, that the function that creates the histogram is rather simple. It takes a NumPy array (`data: np.ndarray`) and keyword arguments (`hist_kws: dict[str, Any]`) for matplotlib's `plt.hist2d()`. But what if we want to use the (x, y) data to plot the a free energy-representation of the 2D latent space. Let's develop such a function. We will use SKLearn's `make_blobs()` function to create the test data.

In [ ]:
from sklearn.datasets import make_blobs
data, categories = make_blobs(n_samples=10_000, n_features=2)
px.scatter(x=data[:, 0], y=data[:, 1])

Next, we will create a function called `to_free_energy` to get the negative log density of a binning of this (x, y) space.

In [ ]:
def to_free_energy(data, bins=100):
    """Adapted from PyEMMA.

    Args:
        data (np.ndarray): The low-dimensional data
            as a NumPy array.
        bins (int): The number of bins.

    Returns:
        tuple[np.ndarray, np.ndarray, np.ndarray]:
            A tuple with the x-centers, the y-centers
            as 1D arrays, and the free energy per bin
            as a 2D array.

    """
    # create a histogram
    H, xedges, yedges = np.histogram2d(*data.T, bins=bins)

    # get the bin centers
    x = 0.5 * (xedges[:-1] + xedges[1:])
    y = 0.5 * (yedges[:-1] + yedges[1:])

    # to density
    density = H / float(H.sum())

    # to free energy
    F = np.inf * np.ones_like(H)
    nonzero = density.nonzero()
    F[nonzero] = -np.log(density[nonzero])

    # shift, so that no zeros are in the data
    F[nonzero] -= np.min(F[nonzero])

    # return
    return x, y, F

Let's test our function:

In [ ]:
xx, yy, z = to_free_energy(data)
fig = px.imshow(z.T, origin="lower", width=500, height=500)
fig.show()

### Provide this function to EncoderMap

We need to make some adjustments to be able to see similar images in tensorboard.

1) Everything needs to be contained in a single function, that takes the low-dimensional output of the encoder as input.
2) The function needs to return a tensorflow image.

Some other lines we have to add:
- `buf = io.BytesIO()`. Raw bytecode buffer. These are the actual bytes that would have ended up on your disk, if you would have written the png to it.

In [ ]:
def free_energy_tensorboard(lowd): 
    # calculate free energy
    H, xedges, yedges = np.histogram2d(*lowd.T, bins=50)
    x = 0.5 * (xedges[:-1] + xedges[1:])
    y = 0.5 * (yedges[:-1] + yedges[1:])
    
    # to density
    density = H / float(H.sum())
    
    # to free energy
    F = np.inf * np.ones(shape=H.shape)
    nonzero = density.nonzero()
    F[nonzero] = - np.log(density[nonzero])
    # shift so no zeros happen
    F[nonzero] -= np.min(F[nonzero])
    
    # actual plotting
    fig = px.imshow(F.T, origin="lower", width=500, height=500)
    
    # BytesIO
    buf = io.BytesIO()
    fig.write_image(buf)
    buf.seek(0)
    
    # tensorflow
    image = tf.image.decode_png(buf.getvalue(), 4) # 4 is due to RGBA colors.
    image = tf.expand_dims(image, 0)
    return image

This function will return a tf.Tensor, that we can now log in tensorboard.

In [ ]:
free_energy_tensorboard(data)

### Train EncoderMap with our new function.

First get the input data

In [ ]:
df = pd.read_csv('asp7.csv')
dihedrals = df.iloc[:,:-1].values.astype(np.float32)
cluster_ids = df.iloc[:,-1].values
print(dihedrals.shape, cluster_ids.shape)
print(df.shape)

Cerate the parameters.

In [ ]:
parameters = em.Parameters(
tensorboard=True,
periodicity=2*np.pi,
n_steps=100,
main_path=em.misc.run_path('runs/custom_images')
)

When calling the method `add_images_to_tensorboard()` we can provide a list of additional functions. That way we can add as much custom functions as we like. In this case, we only add the new `free_energy_tensorboard` function.

In [ ]:
e_map = em.EncoderMap(parameters, dihedrals)
e_map.add_images_to_tensorboard(dihedrals, image_step=1, additional_fns=[free_energy_tensorboard])

In [ ]:
history = e_map.train()

### Output

Start tensorboard locally via:

```bash
$ tensorboard --logdir . --reload_multifile True
```

If you're on Google colab, you can use tensorboard by loading the tensorboard extension:

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir .

Here's what Tensorboard should display:

<img src="custom_images_1.png" width="800">

After training we can use the `to_free_energy()` to plot the latent space after the training is finished.

In [ ]:
px.imshow(to_free_energy(e_map.encode())[-1].T, origin="lower", width=500, height=500)

<a id="writing-custom-callbacks"></a>

## Writing custom callbacks

Writing custom callbacks gives us much more freedom. We can use all kinds of data, that can be provided at the instantiation of a callback. We can also write the images to drive, and so on. We will subclass `encodermap.callbacks.EncoderMapBaseCallback` and implement our own custom functionality in the `on_summary_step()` method. Firt, let's bring up the documentation of that class to see how we can implment our subclass.

In [ ]:
?em.callbacks.EncoderMapBaseCallback

### Polar coordinates

Then, let's come up with something to plot to tensorboard. I've always like polar plots. So we will analyze the output of the EncoderMap model by polar histograms. Our train data allows that, because we are training on the dihedral (torsion) angles of Asp7. So, the output will always be in a certain range. Let's get some data and create our plot without the callback first.

In [ ]:
output = e_map.decode(e_map.encode())
print(output.min())
print(output.max())

We can see, that the output lies within the $(-\pi, \pi)$ periodic space.

In [ ]:
fig = make_subplots(
    cols=2,
    rows=1,
    specs=[[{"type": "polar"}, {"type": "polar"}]],
    subplot_titles=["input", "output"],
)

# input
radii, bins = np.histogram(dihedrals, bins=25)
bins_degree = np.rad2deg(bins)
widths = np.diff(bins)
fig.add_trace(
    go.Barpolar(
        r=radii,
        theta=bins_degree,
    ),
    col=1,
    row=1,
)

# output
radii, bins = np.histogram(output, bins=25)
bins_degree = np.rad2deg(bins)
widths = np.diff(bins)
fig.add_trace(
    go.Barpolar(
        r=radii,
        theta=bins_degree,
    ),
    col=2,
    row=1,
)
fig.update_layout(
    {
        "height": 500,
        "width": 1000,
    },
)
fig.show()

We also can see that this instance of EncoderMap has some success in recreating the distribution of dihedral angles.

### Subclassing `EncoderMapBaseCallback`

We now know, what we want to plot. We just need to implement it.

In [ ]:
class PolarCoordinatesCallback(em.callbacks.EncoderMapBaseCallback):
    # we use our input data as a class attribute
    # (rather than an instance attribute)
    # that way, we can also plot the input diagram
    highd_data = dihedrals.copy()

    def on_summary_step(self, step, logs=None):
        # get output data
        output = self.model(self.highd_data)
        
        fig = make_subplots(
            cols=2,
            rows=1,
            specs=[[{"type": "polar"}, {"type": "polar"}]],
            subplot_titles=["input", "output"],
        )

        # input
        radii, bins = np.histogram(self.highd_data, bins=25)
        bins_degree = np.rad2deg(bins)
        widths = np.diff(bins)
        fig.add_trace(
            go.Barpolar(
                r=radii,
                theta=bins_degree,
            ),
            col=1,
            row=1,
        )
        
        # output
        radii, bins = np.histogram(output, bins=25)
        bins_degree = np.rad2deg(bins)
        widths = np.diff(bins)
        fig.add_trace(
            go.Barpolar(
                r=radii,
                theta=bins_degree,
            ),
            col=2,
            row=1,
        )
        fig.update_layout(
            {
                "height": 500,
                "width": 1000,
            },
        )

        # BytesIO
        buf = io.BytesIO()
        fig.write_image(buf)
        buf.seek(0)
        
        # tensorflow
        image = tf.image.decode_png(buf.getvalue(), 4) # 4 is due to RGBA colors.
        image = tf.expand_dims(image, 0)
        with tf.name_scope("User Provided Plotting Functions"):
            tf.summary.image(f"Polar Plot", image, step=self.steps_counter)

### Adding the callback to EncoderMap

Before starting the training we will simply append use the `add_callback()` method of the `EncoderMap` instance.

In [ ]:
parameters = em.Parameters(
tensorboard=True,
n_steps=100,
periodicity=2*np.pi,
main_path=em.misc.run_path('runs/custom_images')
)

In [ ]:
e_map = em.EncoderMap(parameters, dihedrals)
e_map.add_images_to_tensorboard(dihedrals, image_step=1, additional_fns=[free_energy_tensorboard])

# add the new callback
e_map.add_callback(PolarCoordinatesCallback)

In [ ]:
history = e_map.train()

### Output

The output from Tensorboard could look something like this:

<img src="custom_images_2.png" width="800">

We can clearly see our polar histogram function. Furthermore, we can see, that at training step 10 the EncoderMap network was not yet able to reproduce the input dihedral distribution.

## Conclusion

Using the tools provided in this notebook, you will be able to customize EncoderMap to your liking. Using images to visualize the output of the neural network is a much better visual aid, than just looking at graphs of raw data.